### **ETL (Extract Transform Load)**

Mediante este  notebook se llevará a cabo el proceso de ETL, mediante el cuál se realizarán las transformaciones necesarias a los datos a fines de ser utilizados por las funciones y modelo de recomendación a desarrollar. Una vez hechas las transformacionse y limpieza de datos correspondientes se exportarán en formato csv para ser utilizados por las funciones y el modelo de recomendación a desarrollar.

In [18]:
# Librerias
import pandas as pd
import gzip
import json
import ast
import numpy as np
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import wordcloud
from wordcloud import WordCloud, STOPWORDS
import fastparquet
import pyarrow as pa
import pyarrow.parquet as pq


### *Data Set: Steam_Game*

In [29]:
# Lee linea por linea el archivo steam_games y lo almacena en una lista
ruta= '../STEAM_GAMES_PROYECTOML/DataSets/steam_games.json.gz'
steam_list=[] # lista de dattos

with gzip.open(ruta,'rt',encoding='utf-8') as archivo:
    for fila in archivo:
        datos=json.loads(fila)
        steam_list.append(datos)

# Crear una DataFrame de pandas a partir de la lista de diccionarios steam_list
steam_games=pd.DataFrame(steam_list)
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [33]:
# Crea una copia del datafrae para hacer las transformaciones correspondientes
steam_games_listo=steam_games.copy()
steam_games_listo

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


En base a las funciones y al modelo a desarrollar, solo se trabajara con algunas columnas en especifico, por tal motivo varias de las columnas que no sean reelevantes para el desarrollo de las funciones y del modelo, serán eliminadas.

In [34]:
# Elimina columnas no reelevantes 
steam_games_listo.drop(columns=['url','reviews_url','developer'],inplace=True)
steam_games_listo

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,False,773640
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870


In [35]:
# Elimina registros con valores nulos 

steam_games_listo.dropna(how='all',inplace=True)
steam_games_listo

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,False,767400
88314,NaN,NaN,Log Challenge,NaN,NaN,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,False,773570
...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,False,773640
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870


Se observa que la columna "app_name" y la columna "title" contienen la misma informacion, por tal motivo se opta por eliminar la columna con mayor cantidad de nulos, y dejando vigente aquella que mayor cantidad de datos posea con el fin de evitar la perdida excesiva de datos durante el proceso de transformación.

In [36]:
steam_games_listo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   release_date  30068 non-null  object
 5   tags          31972 non-null  object
 6   specs         31465 non-null  object
 7   price         30758 non-null  object
 8   early_access  32135 non-null  object
 9   id            32133 non-null  object
dtypes: object(10)
memory usage: 2.7+ MB


In [37]:
#Elimina columna "title"
steam_games_listo.drop('title',axis=1,inplace=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,False,767400
88314,NaN,NaN,Log Challenge,NaN,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,False,773570
...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,False,773640
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870


In [48]:
# Elimnar columna "price" ya que no será de utilidad para el desarrollo de las funciones y del modelo de recomendación.
steam_games_listo.dropna(subset='price',inplace=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140
1,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140
2,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140
3,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Casual,Single-player,4.99,False,761140
4,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Simulation,Single-player,4.99,False,761140
...,...,...,...,...,...,...,...,...,...
2088821,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,HTC Vive,4.99,True,681550
2088822,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Oculus Rift,4.99,True,681550
2088823,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Tracked Motion Controllers,4.99,True,681550
2088824,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Standing,4.99,True,681550


#### **Columnas con datos anidados**

Se observa que varias de las columnas del data set están anidadas, por tal motivo se procede a desanidarlas

In [38]:
# Desanida columna "genres"
steam_games_listo=steam_games_listo.explode('genres').reset_index(drop=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140
1,Kotoshiro,Casual,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140
2,Kotoshiro,Indie,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140
3,Kotoshiro,Simulation,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140
4,Kotoshiro,Strategy,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,False,761140
...,...,...,...,...,...,...,...,...,...
74832,Laush Studio,Racing,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660
74833,Laush Studio,Simulation,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660
74834,SIXNAILS,Casual,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870
74835,SIXNAILS,Indie,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870


In [39]:
# Desanida columna "tags"
steam_games_listo=steam_games_listo.explode('tags').reset_index(drop=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Strategy,[Single-player],4.99,False,761140
1,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Action,[Single-player],4.99,False,761140
2,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Indie,[Single-player],4.99,False,761140
3,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Casual,[Single-player],4.99,False,761140
4,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Simulation,[Single-player],4.99,False,761140
...,...,...,...,...,...,...,...,...,...
448163,NaN,NaN,Maze Run VR,NaN,Adventure,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550
448164,NaN,NaN,Maze Run VR,NaN,Indie,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550
448165,NaN,NaN,Maze Run VR,NaN,Action,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550
448166,NaN,NaN,Maze Run VR,NaN,Simulation,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550


In [40]:
# Desanida columna "specs"
steam_games_listo=steam_games_listo.explode('specs').reset_index(drop=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140
1,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140
2,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140
3,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Casual,Single-player,4.99,False,761140
4,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Simulation,Single-player,4.99,False,761140
...,...,...,...,...,...,...,...,...,...
2088821,NaN,NaN,Maze Run VR,NaN,VR,HTC Vive,4.99,True,681550
2088822,NaN,NaN,Maze Run VR,NaN,VR,Oculus Rift,4.99,True,681550
2088823,NaN,NaN,Maze Run VR,NaN,VR,Tracked Motion Controllers,4.99,True,681550
2088824,NaN,NaN,Maze Run VR,NaN,VR,Standing,4.99,True,681550


#### **Tratamiento de Valores nulos**

In [41]:
# Revisar valores nulos 

steam_games_listo.isnull().sum()

publisher       341009
genres           86886
app_name             9
release_date     84407
tags               321
specs             3317
price            68776
early_access         0
id                 201
dtype: int64

Para evitar la perida de datos, para aquellas variables del tipo categoricas que contengan una gran cantidad de nulos, no se optará por la eliminacion de dichos nulos sino por el reemplazo de los mismo por etiquetas tales como "sin_especificar"

In [42]:
# Reemplaza nulos de la coumna "publisher"
steam_games_listo['publisher'].fillna('sin_especificar',inplace=True)

In [43]:
# Reemplaza nulos de la columna "genres"
steam_games_listo['genres'].fillna('sin_especificar',inplace=True)

In [44]:
# Elimina nulos de la columkna "app_name"
steam_games_listo.dropna(subset='app_name',inplace=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140
1,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140
2,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140
3,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Casual,Single-player,4.99,False,761140
4,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Simulation,Single-player,4.99,False,761140
...,...,...,...,...,...,...,...,...,...
2088821,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,HTC Vive,4.99,True,681550
2088822,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Oculus Rift,4.99,True,681550
2088823,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Tracked Motion Controllers,4.99,True,681550
2088824,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Standing,4.99,True,681550


In [45]:
# Elimina nulos de la columna "tags"
steam_games_listo.dropna(subset='tags',inplace=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140
1,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140
2,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140
3,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Casual,Single-player,4.99,False,761140
4,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Simulation,Single-player,4.99,False,761140
...,...,...,...,...,...,...,...,...,...
2088821,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,HTC Vive,4.99,True,681550
2088822,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Oculus Rift,4.99,True,681550
2088823,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Tracked Motion Controllers,4.99,True,681550
2088824,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Standing,4.99,True,681550


In [46]:
# Elimina nulos de la coumna "specs"
steam_games_listo.dropna(subset='specs',inplace=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140
1,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140
2,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140
3,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Casual,Single-player,4.99,False,761140
4,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Simulation,Single-player,4.99,False,761140
...,...,...,...,...,...,...,...,...,...
2088821,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,HTC Vive,4.99,True,681550
2088822,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Oculus Rift,4.99,True,681550
2088823,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Tracked Motion Controllers,4.99,True,681550
2088824,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Standing,4.99,True,681550


In [49]:
# Elimina nulos de la columna "id"
steam_games_listo.dropna(subset='id',inplace=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140
1,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140
2,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140
3,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Casual,Single-player,4.99,False,761140
4,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Simulation,Single-player,4.99,False,761140
...,...,...,...,...,...,...,...,...,...
2088821,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,HTC Vive,4.99,True,681550
2088822,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Oculus Rift,4.99,True,681550
2088823,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Tracked Motion Controllers,4.99,True,681550
2088824,sin_especificar,sin_especificar,Maze Run VR,NaN,VR,Standing,4.99,True,681550


#### **Tratamiento de la columna "release_date"**

Se procederá en primera instancia a cambiar el tipo de dato de esta columna a tipo fecha, luego se creará una columna llamada "release_year" que será de gran utilidad para el desempeño de funciones.

In [52]:
# Cambia tipo de dato, aquellos datos que que no coinsidan con un formato fecha adecuado serán imputados con NaT (nulo)
steam_games_listo['release_date']=pd.to_datetime(steam_games_listo['release_date'],errors='coerce')
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140
1,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140
2,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140
3,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Casual,Single-player,4.99,False,761140
4,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Simulation,Single-player,4.99,False,761140
...,...,...,...,...,...,...,...,...,...
2088821,sin_especificar,sin_especificar,Maze Run VR,NaT,VR,HTC Vive,4.99,True,681550
2088822,sin_especificar,sin_especificar,Maze Run VR,NaT,VR,Oculus Rift,4.99,True,681550
2088823,sin_especificar,sin_especificar,Maze Run VR,NaT,VR,Tracked Motion Controllers,4.99,True,681550
2088824,sin_especificar,sin_especificar,Maze Run VR,NaT,VR,Standing,4.99,True,681550


In [54]:
# Elimina todos aquellos registros cuyo valor en la columna "release_date" sean nulos 
steam_games_listo.dropna(subset='release_date',inplace=True)
steam_games_listo

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Strategy,Single-player,4.99,False,761140
1,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Action,Single-player,4.99,False,761140
2,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Indie,Single-player,4.99,False,761140
3,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Casual,Single-player,4.99,False,761140
4,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,Simulation,Single-player,4.99,False,761140
...,...,...,...,...,...,...,...,...,...
2088773,SIXNAILS,Indie,EXIT 2 - Directions,2017-09-02,Atmospheric,Steam Achievements,4.99,False,658870
2088774,SIXNAILS,Indie,EXIT 2 - Directions,2017-09-02,Atmospheric,Steam Cloud,4.99,False,658870
2088775,SIXNAILS,Indie,EXIT 2 - Directions,2017-09-02,Relaxing,Single-player,4.99,False,658870
2088776,SIXNAILS,Indie,EXIT 2 - Directions,2017-09-02,Relaxing,Steam Achievements,4.99,False,658870


#### **Exportación de datos**

Luego de que los datos fueron transformados, se exportan en formato csv para luego ser consumidos por las funciones y por el modelo de recomendación.

In [57]:
# Exporta data limpia en formato csv 
ruta='Datos_procesados/steam_games_clean.csv'
steam_games_listo.to_csv(ruta,index=False)

### *DataSet: user_reviews*

In [4]:
# Lee el archivo Json linea por linea y devuelve una lista de diccionarios
ruta='../STEAM_GAMES_PROYECTOML/DataSets/user_reviews.json.gz'
reviews_list=[] # lista donde se almacenan los datos 

with gzip.open(ruta,'rt',encoding='utf-8') as archivo:
    for fila in archivo.readlines():
        datos=ast.literal_eval(fila)
        reviews_list.append(datos)


# Crea un DataFrame a partir de la lista de diccionarios obtenidas en el bloque de codigo anterior
user_reviews=pd.DataFrame(reviews_list)
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [5]:
# Creo una copia para comenzar a transformar los datos
user_reviews_listo=user_reviews.copy()
user_reviews_listo

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


#### **Datos anidados**

Producto de la exploración de los datos se puede observar que el campo "reviews" contiene datos anidados, ya que cada campo está compuesto por una lista de diccionarios, donde cada diccionario puede desanidarse como filas y cada clave dentro de los diccionarios pueden normalizarse,permitiendo obtener columnas adicionales.

In [6]:
# Ver el contenido de cada campo de la columna "reviews"
user_reviews_listo['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [7]:
# Desanida las filas del DataFrame
user_reviews_listo=user_reviews_listo.explode('reviews').reset_index(drop=True)
user_reviews_listo

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [8]:
# Normaliza columna "reviews" 
datos_normalizados=pd.json_normalize(user_reviews_listo['reviews'])
datos_normalizados

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
59328,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,,Posted July 20.,,730,No ratings yet,True,:D


In [9]:
# Concatena el DataFrame inicial con los datos normalizados 
user_reviews_listo=pd.concat([user_reviews_listo,datos_normalizados],axis=1)
user_reviews_listo

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la...",,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las...",,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la...",,Posted July 20.,,730,No ratings yet,True,:D


In [10]:
# Elimnar columnas no reelevantes 
user_reviews_listo.drop(columns=['user_url','funny','last_edited','helpful','reviews'],inplace=True)
user_reviews_listo

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


#### **Tratamiendo de valores nulos**

Debido a la cantidad de valores nulos presentes en el DataFrame, se determinó eliminar todos aquellos registros que contengan valores nulos, dado que la cantidad de nulos es irrelevante respecto de la cantidad total de datos presentes en el DataFrame.

In [11]:
# Elimina registros con valores nulos 
user_reviews_listo.dropna(how='all',inplace=True)
user_reviews_listo.reset_index(drop=True)
user_reviews_listo

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


In [12]:
# Ver cantidad de nulos por columnas 
user_reviews_listo.isnull().sum()

user_id       0
posted       28
item_id      28
recommend    28
review       28
dtype: int64

In [13]:
# Elimina valores nulos
user_reviews_listo.dropna(inplace=True)
user_reviews_listo

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


#### **Registros duplicados**

In [14]:
# Ver registros duplicados 
user_reviews_listo[user_reviews_listo.duplicated()]

,user_id,posted,item_id,recommend,review
1114,bokkkbokkk,"Posted September 24, 2015.",346110,True,yep
2894,ImSeriouss,"Posted January 10, 2014.",218620,True,"Good graphics, fun heists! A bit laggy"
2895,ImSeriouss,"Posted January 10, 2014.",105600,True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...
2896,ImSeriouss,"Posted December 17, 2014.",570,True,bobo pinoy
2897,ImSeriouss,"Posted January 13, 2014.",211820,True,If you want to play this game.. expect glithes...
...,...,...,...,...,...
44456,76561198092022514,Posted July 3.,422400,True,Muy entretenido y una coleccion de armas prome...
44457,76561198092022514,Posted June 1.,218620,True,"Tiene una jugabilidad y tematica muy buena :D,..."
44458,76561198092022514,"Posted August 17, 2014.",261820,True,"Buen juego, no importa el desarrrollo que tien..."
44459,76561198092022514,"Posted February 17, 2014.",224260,True,exelente aporte :D¡¡¡ es una buen mod basado e...


In [15]:
# Elimina registros duplicados 
user_reviews_listo.drop_duplicates(inplace=True)
user_reviews_listo

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,True,:D


#### **Tratamiendo de columna "posted"**

La columna "posted" contiene datos acerca de la fecha de publicación de cada una de las reseñas, se observa que dicha columna contiene datos sucios, por tal motivo se procederá a limpiar los datos de esta columna y crear  a partir de ella una nueva columna llamada "year" donde se muestre el año en que se realizó la reseña.

In [16]:
# Eliminar la Palabra "Posted" de la columna  , para ello separ en otra columna la palabra Posted y despues eliminarla 
user_reviews_listo[['Posted','review_posted']]=user_reviews_listo['posted'].str.split('Posted',expand=True)
user_reviews_listo.head(5)

,user_id,posted,item_id,recommend,review,Posted,review_posted
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,,"November 5, 2011."
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,,"July 15, 2011."
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,,"April 21, 2011."
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,,"June 24, 2014."
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,,"September 8, 2013."


In [17]:
# Transforma columna posted a tipo fecha, para los valores erroneos devuleve NaT (nulo)
user_reviews_listo['review_posted']=pd.to_datetime(user_reviews_listo['review_posted'],errors='coerce')
user_reviews_listo

,user_id,posted,item_id,recommend,review,Posted,review_posted
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,,2011-11-05
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,,2011-07-15
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,,2011-04-21
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,,2014-06-24
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,,2013-09-08
...,...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...,,NaT
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...,,NaT
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,,NaT
59331,LydiaMorley,Posted July 20.,730,True,:D,,NaT


In [18]:
# Creacion de la columna year a partir de "review_posted"
user_reviews_listo['year']=user_reviews_listo['review_posted'].dt.year
user_reviews_listo

,user_id,posted,item_id,recommend,review,Posted,review_posted,year
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,,2011-11-05,2011.0
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,,2011-07-15,2011.0
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,,2011-04-21,2011.0
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,,2014-06-24,2014.0
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,,2013-09-08,2013.0
...,...,...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,a must have classic from steam definitely wort...,,NaT,NaN
59329,76561198312638244,Posted July 8.,362890,True,this game is a perfect remake of the original ...,,NaT,NaN
59330,LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,,NaT,NaN
59331,LydiaMorley,Posted July 20.,730,True,:D,,NaT,NaN


In [19]:
# Elimina columnas sobrantes 
user_reviews_listo.drop(columns=['posted','Posted'],inplace=True)
user_reviews_listo

,user_id,item_id,recommend,review,review_posted,year
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05,2011.0
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15,2011.0
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,2011.0
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24,2014.0
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08,2013.0
...,...,...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...,NaT,NaN
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...,NaT,NaN
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,NaT,NaN
59331,LydiaMorley,730,True,:D,NaT,NaN


In [20]:
# Elimina nulos producto de la transformacion anterior 
user_reviews_listo.dropna(inplace=True)
user_reviews_listo

,user_id,item_id,recommend,review,review_posted,year
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05,2011.0
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15,2011.0
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,2011.0
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24,2014.0
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08,2013.0
...,...,...,...,...,...,...
59280,wayfeng,730,True,its FUNNNNNNNN,2015-10-14,2015.0
59283,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10,2015.0
59293,72947282842,730,True,Prettyy Mad Game,2015-10-31,2015.0
59295,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14,2015.0


#### **Feature Engineering**

Se procedera a crear una columna llamada "sentiment_analysis" la cual será producto de un anailsis de sentimiento realizado sobre la columna "reviews", la columna creada adoptará valores 0, 1 y 2, donde 0 corresponderá a reseñas negativas, 1 corresponderá a reseñeas neutrales y 2 correspondera a reseñas positivas.

In [22]:
# Crea funcion que cacule la polaridad y del texto
def polaridad(texto):
    text=TextBlob(texto).sentiment.polarity
    return text

In [23]:
# Crea columna polaridad 
user_reviews_listo['polarity']=user_reviews_listo['review'].apply(polaridad)
user_reviews_listo

,user_id,item_id,recommend,review,review_posted,year,polarity
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05,2011.0,0.174444
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15,2011.0,0.337500
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,2011.0,0.050000
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24,2014.0,0.114583
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08,2013.0,-0.036111
...,...,...,...,...,...,...,...
59280,wayfeng,730,True,its FUNNNNNNNN,2015-10-14,2015.0,0.000000
59283,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10,2015.0,0.300000
59293,72947282842,730,True,Prettyy Mad Game,2015-10-31,2015.0,-0.512500
59295,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14,2015.0,0.100000


In [24]:
# Funcion que asigna valores de sentimiento a la polaridad 
def sentimiento(polaridad):
    if polaridad<0:
        return 0
    elif polaridad>0:
        return 2
    
    elif polaridad==1:
        return 1

    else:
        return 1
    
# Crear columna sentimiento 
user_reviews_listo['sentiment_analysis']=user_reviews_listo['polarity'].apply(sentimiento)
user_reviews_listo

,user_id,item_id,recommend,review,review_posted,year,polarity,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05,2011.0,0.174444,2
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15,2011.0,0.337500,2
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,2011.0,0.050000,2
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24,2014.0,0.114583,2
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08,2013.0,-0.036111,0
...,...,...,...,...,...,...,...,...
59280,wayfeng,730,True,its FUNNNNNNNN,2015-10-14,2015.0,0.000000,1
59283,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10,2015.0,0.300000,2
59293,72947282842,730,True,Prettyy Mad Game,2015-10-31,2015.0,-0.512500,0
59295,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14,2015.0,0.100000,2


In [26]:
user_reviews_listo.drop(columns=['polarity'],inplace=True)

#### **Exportación de datos**

In [113]:
# Exporta datos limpios en formato csv 
ruta='Datos_procesados/user_reviews_clean.csv'
user_reviews_listo.to_csv(ruta,index=False)

### *DataSet: user_items*

In [4]:
# Lee linea por linea el archivo Json y retorna una lista de diccionarios 
ruta='../STEAM_GAMES_PROYECTOML/DataSets/users_items.json.gz'
items_list=[]
no_leidos=[]

with gzip.open(ruta,'rt',encoding='utf-8') as archivo:
    for fila in archivo.readlines():
        datos=ast.literal_eval(fila)
        items_list.append(datos)
        
user_items=pd.DataFrame(items_list)
user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [5]:
#Crea una copia para comenzar con las transformaciones 
user_items_listo=user_items.copy()
user_items_listo

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [6]:
# Muestra registros nulos
user_items_listo.isnull().sum()

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

#### **Columna Items**

In [7]:
# Explorar columna "items"
# la columna contiene una lista de diccionarios, cada diccionario puede desanidarse en 4 columnas [item_id,item_name,playtime_forever,playtime_2weeks]
user_items_listo['items'][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

Luego de realizar la exploración correspondiente se detectó que la columna items se encutra anidada, es por ello que en el siguiente apartado nos concentraremos en desanidar estos datos.

In [8]:
# desanidamos filas 
user_items_listo=user_items_listo.explode('items').reset_index(drop=True)
user_items_listo

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus..."
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To..."
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1..."
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


In [9]:
# desanidamos columnas 
columna_item_deanida=pd.json_normalize(user_items_listo['items'])
columna_item_deanida

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...
5170010,373330,All Is Dust,0.0,0.0
5170011,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,521570,You Have 10 Seconds 2,4.0,4.0
5170013,519140,Minds Eyes,3.0,3.0


In [10]:
# concatena el DataFrame con los datos normalizados 
user_items_listo=pd.concat([user_items_listo,columna_item_deanida],axis=1)
user_items_listo

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus...",373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To...",388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1...",521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes...",519140,Minds Eyes,3.0,3.0


In [11]:
# eliminar columas user_url , items , playtime_2weeks
user_items_listo.drop(columns=['user_url','items','playtime_2weeks'],inplace=True)
user_items_listo

,user_id,items_count,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0
5170011,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0
5170012,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0
5170013,76561198329548331,7,76561198329548331,519140,Minds Eyes,3.0


In [12]:
# Muestra registros duplicados
user_items_listo[user_items_listo.duplicated()]

,user_id,items_count,steam_id,item_id,item_name,playtime_forever
75762,bokkkbokkk,0,76561198006988360,NaN,NaN,NaN
164518,Nikiad,109,76561198084006094,20,Team Fortress Classic,5.0
164519,Nikiad,109,76561198084006094,50,Half-Life: Opposing Force,0.0
164520,Nikiad,109,76561198084006094,70,Half-Life,0.0
164521,Nikiad,109,76561198084006094,130,Half-Life: Blue Shift,0.0
...,...,...,...,...,...,...
4910939,76561198080057659,39,76561198080057659,221910,The Stanley Parable,53.0
4910940,76561198080057659,39,76561198080057659,261030,The Walking Dead: Season Two,253.0
4910941,76561198080057659,39,76561198080057659,273110,Counter-Strike Nexon: Zombies,0.0
4910942,76561198080057659,39,76561198080057659,730,Counter-Strike: Global Offensive,0.0


In [13]:
# Elimina registros duplicados 
user_items_listo=user_items_listo.drop_duplicates().reset_index(drop=True)
user_items_listo

,user_id,items_count,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...,...,...
5110801,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0
5110802,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0
5110803,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0
5110804,76561198329548331,7,76561198329548331,519140,Minds Eyes,3.0


#### **Tratamiento de valores nulos**

En el siguiente apartado puede observrse que el DataSet presenta una gran cantidad de valores nulos, para evitar la perdida masiva de datos, decidi reemplazar aquellos valores nulos de la columna "item_name" por la etiqueta "sin_especificar", luego dedidí eliminar los valores  nulos presentes en la columna "item_id" , respecto a la columna "playtime_forever" tambien decidí eliminar aquellos valores que sean nulos.

In [14]:
# Muestra valores nulos presentes en los datos 
user_items_listo.isnull().sum()

user_id                 0
items_count             0
steam_id                0
item_id             16714
item_name           16714
playtime_forever    16714
dtype: int64

In [15]:
# Reeemplaza nulos de la columna "item_name" por la etiqueta "sin_especificar"
user_items_listo['item_name'].fillna('sin_especificar',inplace=True)

In [16]:
# Elimina registros que contengan valores nulos en las columnas "item_name" y "playtime_forever"
user_items_listo.dropna(inplace=True)
user_items_listo

,user_id,items_count,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...,...,...
5110800,76561198329548331,7,76561198329548331,346330,BrainBread 2,0.0
5110801,76561198329548331,7,76561198329548331,373330,All Is Dust,0.0
5110802,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3.0
5110803,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4.0


#### **Cambiar Tipos**

In [17]:
# Cambiar tipos de datos para las columnas "item_id"  y "playtime_forever"
user_items_listo['item_id']=user_items_listo['item_id'].astype(int)
user_items_listo['playtime_forever']=user_items_listo['playtime_forever'].astype(int)

#### **Exportar Datos**

**Observaciones** 
Pesa mas el archivo exportado en parquet, por lo cual optaré por comprimirlo en gzip

In [22]:
ruta='Datos_procesados/user_items_clean.parquet'
user_items_listo.to_parquet(ruta,compression='gzip')